<h2 align="center">TALLER 5. INTRODUCCIÓN AL ANÁLISIS DE IMÁGEN</h2>

<div style="text-align: justify"> El presente archivo muestra el desarrollo del taller 5. En este taller se desarrolla tanto el ejemplo en clase de los círculos como el análisis de micrografía de una metalografía dada. Así, contenido del el taller 5 acerca de análisis de imágen se organiza de la siguiente manera: </div>

<div style="text-align: left"> 
<h4><a href="#punto1">1. Determinación del diámetro de una vesícula junto con su respectivo centroide</a></h4>
<h4><a href="#punto2">2. Granulometría: Determinación de del tamaño de esféras de grafito en hierro nodular</a></h4>  </div>
<h4><a href="#punto2">3. Bibliografía</a></h4>  </div>


In [159]:
# Librerias importadas para el desarrollo del taller
%pylab inline
import scipy
from scipy import misc #Miscelanea
from scipy import ndimage # Librería Imágenes
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
from PIL import Image
from skimage import feature
import cv2
import seaborn as sns
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


<h4><a name="punto1">1. Determinación del diámetro de una vesícula junto con su respectivo centroide</a></h4>

<div style="text-align: justify"> Tal como aparece en el enunciado, el objetivo de este ejercicio es hacer un reconocimiento de las dimensiones en vesículas lipídicas. Así, el propósito es lograr obtener tanto la posición del centroide de la vesícula como el diámetro de la misma (todo en medidas de pixeles debido a que no se conoce la escala exacta de la figura). Para ello, se utiliza la figura 1 entregada en clase, la cual muestra una fotografía monocromática con ruido de una vesícula lipídica.  </div>


<img style="float: center;" src="ves_full_150_002.jpg" style="height:270px"> </img>
<p></p>
<div style="text-align:center"> <b>Figura 1. Vesícula lipídica dada en clase</b> </div>

<div style="text-align: justify"> Con base en esta figura se procede a realizar el análisis de imágen de la vesícula. Para  ello, se empieza por realizar un filtro gaussiando. Esto con el objetivo de hacer más visible el borde de la vesícula para aplicar el filtro canny. Dicho esto, en la figura 2a se observa el filtro aplicado. En este caso, se utilizó un filtro con $\sigma=1.8$. El valor de este parámetro se obtuvo iterando hasta obtener uno en el que filtro canny (con un parámetro de $\sigma = 3.0$) reconociera el borde del circulo de manera correcta. Cabe aclarar que antes de aplicar los filtros fue necesario pasar la imagen a blanco y negro, lo cual se puede visualizar en el código. Así, la figura 2 muestra el filtro gaussiando y el filtro canny utilizados: </div>

In [2]:
Im1 = pylab.imread("ves_full_150_002.jpg") # Se lee la imagen
vsa = Image.open("ves_full_150_002.jpg") # Se abre la imagen para obtener la matriz RGB de sus colores
gray = vsa.convert('L') #Se convierte la imagen a gris
bw = numpy.asarray(gray) #Se implimenta un array con las propiedades de la imagen en gris
f1 = ndimage.gaussian_filter(bw, 1.8) #Se aplica el filtro gaussiano
bordes = feature.canny(f1, sigma=3) #Se aplica el filtro canny

fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].imshow(f1); axes[0].set_title('a) Filtro gaussiando aplicado ($\sigma$=1.8)')
axes[1].imshow(bordes); axes[1].set_title('b) Filtro Canny ($\sigma = 3.0$)')
plt.close()
fig.savefig('figura2', bbox_inches='tight');

<img style="float: center;" src="figura2.png"> </img>
<p></p>
<div style="text-align:center"> <b>Figura 2. Filtro gaussiano y filtro canny aplicados sobre la vesícula</b> </div>

<div style="text-align: justify"> Posterior a esto, se procede a hacer el filtro hough. Este filtro consiste en iterar valores de una diámetros definidos soobre los gradientes de color de la imagen obtenida después del cannny. Así, se realiza el filtro desde los radios $r_1=210\ ppx$ y $r_2=350\ ppx$. Así, con este filtro se obtiene la imagen completa del borde de la vesícula (figura 3b). Luego de esto, se procede a calcular la posición del centroide (centro_x y centro_y) y se grafica lo obtenido sobre la imagen original. </div>

In [3]:
hough_radii = np.arange(210, 350, 2) #radio iterado mediante el filtro
hough_res = hough_circle(bordes, hough_radii) #filtro utilizado sobre el canny
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,
                                           total_num_peaks=1) #cálculo de los bordes mediante las coordenadas cx y cy
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)

# Cálculo del centro del círculo
centro_x = max(circx)-max(radii)
centro_y = max(circy)-max(radii)


In [4]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 4),
                                    sharex=True, sharey=True)
ax1.imshow(Im1, cmap=plt.cm.gray)
ax1.set_title('a) Imagen original', fontsize=10); ax1.axis('off')
ax2.plot(circx,circy, ',m',lw=0.5); ax2.axis('off')
ax2.set_title('b) Bordes encontrados con el filtro Hough', fontsize=10)
ax2.plot(centro_x,centro_y,'om')
ax3.imshow(Im1); axis('off')
ax3.plot(circx,circy, ',m',lw=0.5)
ax3.plot(centro_x,centro_y,'om')
ax3.set_title('c) Superposición de las dos imágenes', fontsize=10)
print(f'Diámetro [píxeles] = {max(radii)}')
print(f'Centro [píxeles] = {[centro_y, centro_x]}')

fig.tight_layout()

plt.close()
fig.savefig('figura3', bbox_inches='tight');

Diámetro [píxeles] = 238
Centro [píxeles] = [280, 276]


<img style="float: center;" src="figura3.png"> </img>
<p></p>
<div style="text-align:center"> <b>Figura 3. Aplicación del filtro Hough para hallar el borde completo de la vesícula junto con su centroide.</b> </div>

<div style="text-align: justify"> Con base en lo anterior, al observar la figura 3c es posible inferir que los radios elegidos para la iteración fueron correctos, al igual que los parámetros $\sigma$ de los filtros gaussiando y canny. Así, se puede observar que el borde obtenido con el filtro hough se ajusta de una manera bastante precisa al borde real de la imagen, presentando algunas desviaciones cerca al borde derecho de la figura. Para mejorar la precisión, se podría hacer los siguiente:a) agregar otro tipo de filtros como un aumento de contraste y brillo que permitan evitar estos errores; b) variar nuevamente los parámetros $\sigma$ y los radios hasta encontrar los que se aproximen más a la imagen real. </div>

<h4><a name="punto2">2. Granulometría: Determinación del tamaño de inclusiones de grafito en hierro nodular</a></h4>

<div style="text-align: justify"> Luego de haber realizado el filtro hough con una sola imagen, lo siguiente es aplicarlo a un caso de ingeniería. De esta manera, el procedimiento anterior se realizará nuevamente, pero esta vez para una granulometría. De esta manera, la imagen a utilizar es un hierro nodular fundido cuya superficie ha sido pulida, tal como se observa en la figura 4a. Así, el objetivo del ejercicio es encontrar el tamaño de las inclusiones de carbono sobre la superficie en una de sus formas alotrópicas más conocidas: grafito. Con estos resultados se procederá a realizar un histograma de dicho tamaño.Teniendo en cuenta esta imagen, se procede a pasar el formato de colores a un formato binario, de tal forma que la imagen solo quede con dos colores (figura 4b).  </div>

In [51]:
fe_nodular = cv2.imread('fe_nod.JPG')
nod_bin = cv2.cvtColor(fe_nodular,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(nod_bin,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].imshow(fe_nodular); axes[0].set_title('a) Imagen original de hierro nodular pulido a escala 100x')
axes[1].imshow(thresh); axes[1].set_title('b) Imagen obtenida en formato de colores binario')
plt.close()
fig.savefig('figura4', bbox_inches='tight');

<img style="float: center;" src="figura4.png"> </img>
<p></p>
<div style="text-align:center"> <b>Figura 4. Imagen de la metalografía de un hierro nodular pulido en formato de colores original y en formato binario [1]. Este hierro fundido tiene una composición de (Fe-3.6%C-2.9%Si-0.14%Mn-0.04%P-0.02%S-0.16%Ni-0.06%Mg)</b> </div>

<div style="text-align: justify"> Posterior a esto, se procede a eliminar el ruido de la imagen. En este caso, se considera ruido a todas aquellas zonas y puntos de color negro que no tienen el tamaño apropiado para hacer parte del análisis. Para saber esto, se itera el parámetro de la transformada sobre la línea 8 del código mostrado [2]. Así, se obtuvo un parámetro de 0.5 para eliminar el ruido, obteniendo dos imágenes: una imagen más limpia (figura 5a) y una con todos sus bordes delimitados (figura 5b).</div>

In [48]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.4*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

In [49]:
fig, axes = plt.subplots(1, 2, figsize=(15,5))
axes[0].imshow(sure_fg); axes[0].set_title('a) Imagen con ruido filtrado')
axes[1].imshow(unknown); axes[1].set_title('b) Imagen con los bordes demarcados')
plt.close()
fig.savefig('figura5', bbox_inches='tight');

<img style="float: center;" src="figura5.png"> </img>
<p></p>
<div style="text-align:center"> <b>Figura 5. Imagenes de la metalografía luego de remover el ruido sobre la figura 4b y delimitar los bordes de las inclusiones de carbono</b> </div>


<div style="text-align: justify"> A partir de lo anterior se procede a superponer la imagen 5b con la imagen original con el fin de observar la precisión con la cual se estan delimitando los bordes de las inclusiones. Esto se puede observar en la figura 6.</div>

In [52]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0
markers = cv2.watershed(fe_nodular,markers)
fe_nodular[markers == -1] = [255,0,0]
plt.imshow(fe_nodular)
plt.savefig('figura6', bbox_inches='tight');
plt.close()

<img style="float: center;" src="figura6.png" style="height:350px">  </img>
<p></p>
<div style="text-align:center"> <b>Figura 6. Imagen de la metalografía del hierro nodular con las inclusiones de grafito delimitadas </b> </div>

<div style="text-align: justify"> Como se puede observar, los limites son bastante precisos, por lo cual se procede a calcular el tamaño de las inclusiones. Para ello, primero se procede preparar la imagen con la que se va a realizar el filtro hough. En este caso, se quita el color de la figura 6 y se deja solo los bordes delimitados sobre la imagen. Luego, se procede a realizar el filtro hough para esta última imagen (figura 7). Los parámetros del filtro se calcularon iterando los valores, de tal forma que se lograra obtener una aproximación aceptable para calcular el tamaño de las inclusiones mediante el conteo de las mismas. </div>

In [172]:
fe_nodular[markers == -1] = [255,255,255]; fe_nodular[markers != -1] = [0,0,0]
np.save('inclusiones',fe_nodular); hf = np.load('inclusiones.npy')
hf = color.rgb2grey(hf)

hough_radii = np.arange(6, 35, 1)
hough_res = hough_circle(hf, hough_radii)
accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii,total_num_peaks=120)
fe = cv2.imread('fe_nod.JPG')
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10)); completa = color.gray2rgb(fe)
for center_y, center_x, radius in zip(cy, cx, radii):
    circy, circx = circle_perimeter(center_y, center_x, radius)
    completa[circy, circx] = (220, 20, 255)    

ax.imshow(completa)
plt.savefig('figura7', bbox_inches='tight');
plt.close()

<img style="float: center;" src="figura7.png" style="height:450px">  </img>
<p></p>
<div style="text-align:center"> <b>Figura 7. Imagen de la metalografía del hierro nodular con las inclusiones de grafito delimitadas luego del filtro hough </b> </div>

<div style="text-align: justify"> Al observar la figura 7 es posible decir que, a pesar de que la aproximación realizada mediante el filtro hough fue aceptable, no fue posible demarcar los limites de todas las inclusiones de grafito. Esto se debe a que no todas las inclusiones son perfectamente circulares, lo cual hace que el filtro no los reconozca correctamente. Adicional a esto, es posible decir que hay algunos límites de inclusiones que se repiten (círculos concéntricos sobre la misma inclusión). Esto significa que al filtro le tomó multiples iteraciones para reconocer inclusiones que no eran tan circulares, de tal forma que la siguiente mejor iteración antes de reconocer estas inclusiones era aquella que estaba cercana a un círculo ya reconocido. Debido a esto, el histograma realizado a continuación (figura 8) va a presentar un pequeño error ya que incluye partículas de grafito que no existen. Con base en esto, se realiza el histograma correspondiente. Para ello, se utiliza la escala de $100\ \mu m$ que se puede apreciar en la imagen original del hierro nodular para convertir los pixeles. Así, la conversión de unidades es la siguiente:</div>

<div style="text-align: center"> $1\ pixel*\frac{100\ \mu m}{58\ pixeles} = 1.72\frac{\mu m}{pixeles}\ \longrightarrow 1\ pixel = 1.72\ \mu m $ </div>


<div style="text-align: justify">Con este factor de conversión, se realiza histograma correspondiente.</div>

In [176]:
diametros = 2*radii*(100/58);
ax = sns.distplot(diametros, color='purple', kde=False, rug=True)
plt.xlabel("Tamaño de inclusión encontrada $(\mu m)$"); plt.ylabel("Frecuencia relativa")
plt.grid('on')
plt.savefig('figura8', bbox_inches='tight'); plt.close();

<img style="float: center;" src="figura8.png">  </img>
<p></p>
<div style="text-align:center"> <b>Figura 8. Histograma obtenido de los tamaños de las inclusiones de grafito</b> </div>

<div style="text-align: justify">Al analizar el histograma obtenido, es posible decir que, la mayor frecuencia relativa de los granos se encuentra aproximadamente entre los tamaños 26 $\mu m$ y  30 $\mu m$. Con base en esto, es posible decir que esta frecuencia coincide con lo visto en la figura 4a ya que es posible ver que la mayor cantidad de inclusiones no son de gran tamaño, sino más bien pequeño. A pesar de esto, al analizar la cantidad de circulos encontrados con el filtro hough con respecto al número de inclusiones, es posible observar que este resultado varía bastante. Mientras que con el filtro hough se contaron 120 círculos (peaks=120) para lograr delimitar la mayor cantidad de inclusiones posible (sin importar si se repetían ciertos círculos), el número real de inclusiones contadas es de 78. Así, es posible ver que, como si dijo anteriormente, el filtro reconoció más granos de los que realmente habían, lo cual hace que el histograma presente un error dentro de la distribución del tamaño. </div>


<h4><a name="punto3">3. Bibliografía</a></h4>

[1] American Society of Metals, ASM Handbook Volume 9: Metallography and Microstructures, ASM, 2004. 

[2] OpenCV, «Image Segmentation with Watershed Algorithm,» [En línea]. Available: https://docs.opencv.org/3.3.1/d3/db4/tutorial_py_watershed.html. [Último acceso: 15 Abril 2018].